# Rolling sum averages and standard deviations 

Import the necessary libraries and set the variables

In [1]:
import pandas as pd
import numpy as np
# This sets the maximum number of rows that can be 
# printed to the screen
pd.set_option("display.max_rows", 500)

# ------- Set all necessary variables ---------------- #

# Working from within the directory where your excel file is located
infile = "12th_and_13th_trial_unseeded_with_BW_CF_averaging.xlsx"
outfile = "new.xlsx"
col_list = ["Time [h]", "Concentration Factor [-]"]
column2aggregate = "Concentration Factor [-]"
aggregate_functions = ['mean','std']
rolling_interval  = 1600
num_of_rows2view = 30

 Import the tables from Excel

In [2]:
# --- Read in the xlsx file as a pandas dataframe with choice columns --- #
# Read a particular sheet from the excel file
df12 = pd.read_excel(infile, '12th trial with BW', header=1, usecols=col_list)
df13 = pd.read_excel(infile, '13th trial with BW', header=1, usecols=col_list)

# Uncomment the lines below if you previously saved your dataframes as pickles
# df12 = pd.read_pickle('df12.pickle')
# df13 = pd.read_pickle('df13.pickle')

Create a function that will get the list of row indices to clear from the dataframes

In [3]:
# A function to get the list of row indices to clear
def get_indices(interval: int, length: int) -> list:
    """ A function to get the list of row indices to clear 
        in a dataframe
        Example:
        get_indices(interval=100, length=n500)
    """
    indices = np.arange(length+1)
    to_remove = [interval-1]
    right = list(range(to_remove[-1],length,interval))
    to_remove.extend(right)
    indices_to_remove = [np.where(indices == i) for i in to_remove]
    # Here's the game changer, used numpy's ufunc instead of a for loop
    indices = list(np.delete(indices, indices_to_remove))
            
    return indices

Get the indices of cells to be cleared using the function created above. 
**The smaller the rolling interval specified, the longer it takes for this step to complete**

In [4]:
# Number of rows / observations
nrows = len(df12)
indices2remove = get_indices(interval=rolling_interval, length=nrows)

# If the last index is greater than the 
# the available indices rcolnames it
if indices2remove[-1] >= nrows:
    indices2remove.pop()

Calculate the rolling means and standard deviations then delete the unwanted cells

In [5]:
colnames = [f"avg-{rolling_interval}", f"std-{rolling_interval}"]
df12[colnames] =  (df12.rolling(rolling_interval)[column2aggregate]
                                .agg(aggregate_functions))
                                
df12.loc[indices2remove,colnames] = ''
df12.head(num_of_rows2view)

,Time [h],Concentration Factor [-],avg-1600,std-1600
0,0.01168,1.00009,,
1,0.01196,1.00011,,
2,0.01224,1.00016,,
3,0.01254,1.00032,,
4,0.01279,1.00049,,
5,0.01308,1.00068,,
6,0.01336,1.00085,,
7,0.01367,1.00113,,
8,0.01392,1.00130,,
9,0.01421,1.00148,,


Get the indices of cells to be cleared using the function created above. 

**The smaller the rolling interval specified, the longer it takes for this step to complete**

*You can skip this step to save time if the number of rows of df12 is equal to the numebr of rows of df13*

In [6]:
# Number of rows / observations
nrows = len(df13)
indices2remove = get_indices(interval=rolling_interval, length=nrows)

# If the last index is greater than the 
# the available indices remove it
if indices2remove[-1] >= nrows:
    indices2remove.pop()

Calculate the rolling means and standard deviations then delete the unwanted cells

In [7]:
df13[colnames] =  (df13.rolling(rolling_interval)[column2aggregate]
                                .agg(aggregate_functions))
df13.loc[indices2remove,colnames] = ''

df13.head(num_of_rows2view)

,Time [h],Concentration Factor [-],avg-1600,std-1600
0,0.011681,1.00006,,
1,0.011972,1.00009,,
2,0.012236,1.00020,,
3,0.012514,1.00032,,
4,0.012806,1.00053,,
5,0.013097,1.00069,,
6,0.013375,1.00092,,
7,0.013653,1.00110,,
8,0.013931,1.00133,,
9,0.014208,1.00149,,


Write the modified tables to an excel file

In [8]:
# Write to excel
with pd.ExcelWriter(outfile, engine='openpyxl') as writer:
    df12.to_excel(writer, sheet_name="df12", index=False)
    df13.to_excel(writer, sheet_name="df13", index=False)

Just a demo to see how verbose ouputs are printed

In [10]:
# Uncomment to save table for ease of reading next time
# df12.to_pickle('df12.pickle')
# df13.to_pickle('df13.pickle')
help(get_indices)

Help on function get_indices in module __main__:

get_indices(interval: int, length: int) -> list
    A function to get the list of row indices to clear 
    in a dataframe
    Example:
    get_indices(interval=100, length=n500)

